In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
import numpy as np
import time

In [7]:
def findUS (price):
    for tr in price:
        if tr.find('img').next_sibling[1:-1] == 'U.S. Dollar':
            return tr
    return 0

In [96]:
def fixDiscount(string):
    price = string[1].contents[0]
    if price == 'N/A':
        return float(0)
    if price[-3:] == 'at ':
        aux = float(string[1].find('span').contents[0].strip('%'))/100
        price = float(price[1:-3])
        return price /(1+aux)
    return float(price[1:])

In [210]:
def findDeveloper(prod):
    aux = 0
    for i in prod[:len(prod)-1]:
        string = i.findAll('td')[0].contents[0]
        if string == 'Developer':
            return (aux, 1)
        aux = aux + 1
    return (0, 0)

In [211]:
def findPublisher(prod, dev):
    aux = int(dev) + 1
    for i in prod[dev + 1:]:
        string = i.findAll('td')[0].contents[0]
        if string == 'Publisher':
            return (aux, 1)
        aux = aux + 1
    return (0,0)

In [290]:
def scrapePage(url):
    url = "https://steamdb.info/app/" + str(url) + '/'
    headers = {
    'User-Agent': 'SteamDB-Educational-Access; gasouza@telecom-paris.fr',
    'From': 'gasouza@telecom-paris.fr',
    "Accept-Language" : "en-US"}
    
    try:
        page = requests.get(url, headers = headers)
    except requests.exceptions.RequestException as e: 
        return (0, 0)
    
    bs = BeautifulSoup(page.text, 'html.parser')
    
    entry = []
    
    #getting the title
    title = bs.find('h1', itemprop='name')
    title = title.contents[0]
    entry.append(title)
    
    #getting review score
    rs = bs.find('div', 'header-thing-number')
    if (rs.contents[0])[-1]!='%':
        return (2, 0)
    rs = float((rs.contents[0])[2:].strip('%'))
    entry.append(rs)
    
    #getting price
    price = bs.find('div', 'tab-pane selected')
    price = price.find('div', 'table-responsive')
    if price is None:
        price = float(0)
    else:
        price = price.find('tbody')
        price = price.findAll('tr')

        price = findUS(price)
        price = price.findAll('td')
        price = fixDiscount(price)
    entry.append(price)
    
    #getting publisher and developer
    prod = bs.find('table', "table table-bordered table-hover table-dark table-responsive-flex")
    prod = prod.findAll('tr')
    devint, auxD= findDeveloper(prod)
    if auxD==0:
        dev= 'no developer'
        pub='no publisher'
    else:
        dev = (prod[devint].findAll('td')[1].find('a').contents[0])[:-1]
        prodint, auxP= findPublisher(prod, devint)
        if auxP==0:
            pub = 'no publisher'
        else:
            pub = (prod[prodint].findAll('td')[1].find('a').contents[0])[:-1]

    entry.append(dev)
    entry.append(pub)
    
    #getting tags
    tags = bs.findAll('a', 'btn btn-sm btn-outline btn-tag')
    aux = ""
    for i in tags:
        aux = aux + i.contents[0] +  ", "
    entry.append(aux[:-2])
    
    #print(entry)
    return (1, entry)

In [291]:
a, b = scrapePage(368340)

In [10]:
db = sqlite3.connect('games.db')
cursor = db.cursor()

rows = cursor.execute("SELECT app_id FROM GAMES").fetchall()
rows = np.array(rows)
appids = rows[:, 0]

In [5]:
#create a database here so that the cell block below only has to open it and add rows. 
#Ideally I only run this cell once

db = sqlite3.connect('games.db')
db.execute("ALTER TABLE GAMES ADD price FLOAT")
db.execute("ALTER TABLE GAMES ADD developer TEXT")
db.execute("ALTER TABLE GAMES ADD publisher TEXT")
db.execute("ALTER TABLE GAMES ADD tags TEXT")
db.commit()

In [ ]:
#open the database and start from the last row obtained
db = sqlite3.connect('games.db')
i=6048
while i < len(appids):
    aux=i
    #I didnt use the correct function name because if you run this it will start to scrape 
    #like crazy and we'll get banned lol
    error, arr = scrapePage(appids[i])
    if error == 0:
        print("Error connecting to server")
        #there was an error connecting to the website
        #close your database please so we dont lose anything
        time.sleep(10)
        i = i-1
    elif error==2:
        pass
    else:
        #add row to database, information is stored in array
        data_tuple = (arr[2], arr[3], arr[4], arr[5], int(appids[i]))
        db.execute('''UPDATE GAMES
                        SET price = ?,
                            developer = ?,
                            publisher = ?,
                            tags = ?
                        WHERE
                            app_id = ?;''', data_tuple)
        db.commit()
        print(f'Succesfully added: {arr[0]}')
        time.sleep(1)
    i = i+1

Succesfully added: CrossCode
Succesfully added: 60 Seconds!
Succesfully added: Her Story
Succesfully added: The Darkside Detective
Succesfully added: Through the Woods
Succesfully added: Rememoried
Succesfully added: Assassin's Creed Syndicate
Succesfully added: Beat Da Beat
Succesfully added: Albert and Otto: The Adventure Begins
Succesfully added: Terra Nova: Strike Force Centauri
Succesfully added: The Enchanted Cave 2
Succesfully added: Timespinner
Succesfully added: htoL#NiQ: The Firefly Diary
Succesfully added: Dark Echo
Succesfully added: Super Mutant Alien Assault
Succesfully added: Chronoclysm
Succesfully added: Mysterious Space
Succesfully added: Power of Love
Succesfully added: Sirius Online
Succesfully added: No Time To Explain Remastered
Succesfully added: Obliteracers
Succesfully added: Playing History 3 - Vikings
Succesfully added: President for a Day - Floodings
Succesfully added: President for a Day - Corruption
Succesfully added: Apocalypse: Party's Over
Succesfully a

Succesfully added: Ryzom
Succesfully added: Way of Gold and Steel
Succesfully added: DinoSystem
Succesfully added: LiEat
Succesfully added: Fantastic Pinball Thrills
Succesfully added: Hot Pinball Thrills
Succesfully added: Soccer Pinball Thrills
Succesfully added: Apothecarium: The Renaissance of Evil - Premium Edition
Succesfully added: Sister’s Secrecy: Arcanum Bloodlines - Premium Edition
Succesfully added: Fleeting Ages
Succesfully added: Eisenhorn: XENOS
Succesfully added: Sudden Strike 4
Succesfully added: Letter Quest: Grimm's Journey Remastered
Succesfully added: Metrico+
Succesfully added: Sherlock Holmes Consulting Detective: The Case of the Mummy’s Curse
Succesfully added: Virginia
Succesfully added: Portal Knights
Succesfully added: Infinium Strike
Succesfully added: Stairs
Succesfully added: Falcon
Succesfully added: Forktuck Challenge
Succesfully added: Concealed Intent
Succesfully added: Sniper Tactical
Succesfully added: Eternal Step
Succesfully added: Hired Ops
Succes

Succesfully added: Mahjong Pretty Girls Battle : School Girls Edition
Succesfully added: Panzermadels: Tank Dating Simulator
Succesfully added: Hush
Succesfully added: MiniOne Racing
Succesfully added: Three Heroes
Succesfully added: Flute Master
Succesfully added: Celia's Quest
Succesfully added: The Secret of Tremendous Corporation
Succesfully added: STASIS
Succesfully added: Signal to Noise
Succesfully added: Hover Junkers
Succesfully added: Tap Tap Infinity
Succesfully added: Inevitability
Succesfully added: Incandescent
Succesfully added: Spandex Force: Champion Rising
Succesfully added: Axion
Succesfully added: SpaceCorn
Succesfully added: Fishing Planet
Succesfully added: Killing Time at Lightspeed: Enhanced Edition
Succesfully added: Final Approach
Succesfully added: Don'Yoku
Succesfully added: Zombie Killtime
Succesfully added: Anima Gate of Memories
Succesfully added: Herald: An Interactive Period Drama
Succesfully added: Teeworlds
Succesfully added: Barbara-ian
Succesfully a

Succesfully added: Scott in Space
Succesfully added: Steambirds Alliance
Succesfully added: Proto Raider
Succesfully added: Planetary Annihilation: TITANS
Succesfully added: Pharaonic
Succesfully added: Nandeyanen!? - The 1st Sûtra
Succesfully added: Pylon: Rogue
Succesfully added: Raywin
Succesfully added: After The End: The Harvest
Succesfully added: Vampire of the Sands
Succesfully added: Crossout
Succesfully added: STLD Redux: Episode 02
Succesfully added: Microcosmum: survival of cells
Succesfully added: Vendetta - Curse of Raven's Cry
Succesfully added: Kumoon : Ballistic Physics Puzzle
Succesfully added: Naninights
Succesfully added: HoD: On open seas
Succesfully added: MegaSphere
Succesfully added: Rise: Battle Lines
Succesfully added: SMITE
Succesfully added: Foosball: World Tour
Succesfully added: Job the Leprechaun
Succesfully added: Blood Code
Succesfully added: Wanda
Succesfully added: Pyramid Raid
Succesfully added: Champion of the Gods
Succesfully added: A.I. Invasion
Su

Succesfully added: Uebergame
Succesfully added: Evolution
Succesfully added: All Fall Down
Succesfully added: Super Mega Bob
Succesfully added: TRON RUN/r
Succesfully added: RFLEX
Succesfully added: Murasaki
Succesfully added: War Blade
Succesfully added: Pumped BMX +
Succesfully added: Wayward Terran Frontier: Zero Falls
Succesfully added: Fatal Theory
Succesfully added: Endless Space® 2 - Digital Deluxe Edition
Succesfully added: City Z
Succesfully added: #SkiJump
Succesfully added: X4: Foundations
Succesfully added: #SelfieTennis
Succesfully added: Rogue Wizards
Succesfully added: The Franz Kafka Videogame
Succesfully added: Take Command - 2nd Manassas
Succesfully added: Wish -tale of the sixteenth night of lunar month-
Succesfully added: ToyOdyssey
Succesfully added: Silver Creek Falls - Chapter 1
Succesfully added: NOBUNAGA'S AMBITION: Sphere of Influence
Succesfully added: WARSHIFT
Succesfully added: Volvox
Succesfully added: Tricone Lab
Succesfully added: Lucadian Chronicles
Suc

...

Succesfully added: The Sea Eternal
Succesfully added: Storm of Spears
Succesfully added: One Thousand Lies
Succesfully added: Spellbound
Succesfully added: Panic Pump - Can you save them ALL?
Succesfully added: Ironguard
Succesfully added: Retool
Succesfully added: Resette's Prescription ~Book of memory, Swaying scale~
Succesfully added: Lithium: Inmate 39
Succesfully added: A Shooty Bit
Succesfully added: Norman's Great Illusion
Succesfully added: Empires of the Undergrowth
Succesfully added: StreetCraft
Succesfully added: Runeyana
Succesfully added: Super Cloudbuilt 
Succesfully added: Puzzle Galaxies
Succesfully added: The Beggar's Ride
Succesfully added: FreeHolder
Succesfully added: Izeriya
Succesfully added: Muddy Heights 2
Succesfully added: Shadows of Kurgansk
Succesfully added: Deathwave
Succesfully added: Beyond Magic
Succesfully added: Initia: Elemental Arena
Succesfully added: The Dweller
Succesfully added: Solitairica
Succesfully added: Arcade Saga
Succesfully added: Super

Succesfully added: Genital Jousting
Succesfully added: Fable Fortune
Succesfully added: Space Codex
Succesfully added: hackmud
Succesfully added: Pandum online
Succesfully added: Legend of Miro
Succesfully added: NEKOPALIVE
Succesfully added: SnarfQuest Tales, Episode 1: The Beginning
Succesfully added: Just Bones
Succesfully added: Spellgear
Succesfully added: 1917 - The Alien Invasion DX
Succesfully added: Quantum Replica
Succesfully added: Eclipse --- Defending the motherland
Succesfully added: Funklift
Succesfully added: Evangeline™
Succesfully added: Rocketbirds 2 Evolution
Succesfully added: UNO
Succesfully added: Event[0]
Succesfully added: Tribal Pass
Succesfully added: Lost Route
Succesfully added: Winged Sakura: Demon Civil War
Succesfully added: TROUBLESHOOTER: Abandoned Children
Succesfully added: Matris
Succesfully added: Where's My Helmet?
Succesfully added: Birdsketball
Succesfully added: Angel Express [Tokkyu Tenshi]
Succesfully added: RollerGirls From Beyond
Succesfull

Succesfully added: Winter Novel
Succesfully added: Disgraced
Succesfully added: Mad Combat Marines
Succesfully added: Welcome to the Game
Succesfully added: Machine Made: Rebirth
Succesfully added: NAL Is Alive
Succesfully added: Call Of The Mighty Warriors
Succesfully added: Ninja Stealth
Succesfully added: The Banner Saga 3
Succesfully added: Dungeon Journey
Succesfully added: Smithy
Succesfully added: Nioh: Complete Edition
Succesfully added: POPixel
Succesfully added: Milford Heaven - Luken's Chronicles
Succesfully added: Citadel
Succesfully added: OESE
Succesfully added: Ball 3D: Soccer Online
Succesfully added: Pick a Hero
Succesfully added: Brazed
Succesfully added: Genius Greedy Mouse
Succesfully added: Campaign Clicker
Succesfully added: Karma Miwa
Succesfully added: Mini Golf Arena
Succesfully added: sphereFACE
Succesfully added: Hope Lake
Succesfully added: Shift
Succesfully added: Magnetta
Succesfully added: Legend of Numbers
Succesfully added: JOLT: Super Robot Racer
Succe

Succesfully added: Reset 1-1
Succesfully added: Assault on the Necrospire
Succesfully added: Stratego® Single Player
Succesfully added: Gotta Go
Succesfully added: Gun Range VR
Succesfully added: Nary
Succesfully added: Deformers
Succesfully added: Project Syria
Succesfully added: Across The Line
Succesfully added: Glow
Succesfully added: Orwell
Succesfully added: Vision of Aurora Borealis
Succesfully added: A-Train PC Classic / みんなのA列車で行こうPC
Succesfully added: RPG World - Action RPG Maker
Succesfully added: 3D Pool
Succesfully added: Planet Driller
Succesfully added: Haimrik
Succesfully added: Bohemian Killing
Succesfully added: Space Hulk: Tactics
Succesfully added: The Cat! Porfirio's Adventure
Succesfully added: The Agency: Chapter 1
Succesfully added: Has-Been Heroes
Succesfully added: Pixel Cup Soccer 17
Succesfully added: Affairs of the Court: Choice of Romance
Succesfully added: Thunder Gun: Revenge of the Mutants
Succesfully added: Rogues Like Us
Succesfully added: Alteric
Suc

Succesfully added: Crystal Quest Classic
Succesfully added: Crimson Sword Saga: The Peloran Wars
Succesfully added: Building the Great Wall of China 2
Succesfully added: Agent Walker: Secret Journey
Succesfully added: Sapper's bad dream
Succesfully added: Trick & Treat
Succesfully added: LineDash
Succesfully added: Plates
Succesfully added: Safety First!
Succesfully added: Holobunnies: Pause Cafe
Succesfully added: Funfair Ride Simulator 3
Succesfully added: The Wardrobe
Succesfully added: Recursed
Succesfully added: Golden Krone Hotel
Succesfully added: Quanero
Succesfully added: 8-Bit Hordes
Succesfully added: Zezenia Online
Succesfully added: Doodle WHAT?!
Succesfully added: Destiny of Ancient Kingdoms™
Succesfully added: Mushroom Crusher Extreme
Succesfully added: Barnyard Mahjong 3
Succesfully added: The Hidden Dragon
Succesfully added: Mystery Of Rivenhallows
Succesfully added: Dragon Rage
Succesfully added: Batman - The Telltale Series
Succesfully added: Jurassic Survival
Succes

In [292]:
#print(type(np.where(appids==aux)[0][0]))
appids[i]

368340

In [293]:
i

6048